In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split 
import keras 
from keras.models import Sequential 
from keras.layers import InputLayer 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.constraints import maxnorm
from sklearn.preprocessing import LabelEncoder

In [13]:
df = pd.read_csv('bank_data_nan_imputed.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355190 entries, 0 to 355189
Data columns (total 94 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   355190 non-null  int64  
 1   CR_PROD_CNT_IL               355190 non-null  int64  
 2   AMOUNT_RUB_CLO_PRC           355190 non-null  float64
 3   TURNOVER_DYNAMIC_IL_1M       355190 non-null  float64
 4   CNT_TRAN_AUT_TENDENCY1M      355190 non-null  float64
 5   SUM_TRAN_AUT_TENDENCY1M      355190 non-null  float64
 6   AMOUNT_RUB_SUP_PRC           355190 non-null  float64
 7   SUM_TRAN_AUT_TENDENCY3M      355190 non-null  float64
 8   CLNT_TRUST_RELATION          355190 non-null  int64  
 9   REST_DYNAMIC_FDEP_1M         355190 non-null  float64
 10  CNT_TRAN_AUT_TENDENCY3M      355190 non-null  float64
 11  APP_MARITAL_STATUS           355190 non-null  int64  
 12  REST_DYNAMIC_SAVE_3M         355190 non-null  float64
 13 

In [53]:
df.drop(['ID', 'APP_DRIVING_LICENSE', 'APP_TRAVEL_PASS', 'APP_CAR', 'APP_DRIVING_LICENSE'], axis=1, inplace=True)
df.drop(['PRC_ACCEPTS_A_EMAIL_LINK', 'PRC_ACCEPTS_A_POS', 'PRC_ACCEPTS_A_TK', 'PRC_ACCEPTS_A_AMOBILE', 'PRC_ACCEPTS_TK'], axis=1, inplace=True)
df.drop(['PRC_ACCEPTS_A_MTP', 'PRC_ACCEPTS_A_MTP', 'CNT_ACCEPTS_TK', 'PRC_ACCEPTS_A_ATM', 'PRC_ACCEPTS_MTP', 'CNT_ACCEPTS_MTP'], axis=1, inplace=True)
df.drop(['LDEAL_DELINQ_PER_MAXYQZ', 'MED_DEBT_PRC_YQZ', 'LDEAL_DELINQ_PER_MAXYWZ'], axis=1, inplace=True)
df.drop(['LDEAL_ACT_DAYS_PCT_TR3', 'LDEAL_ACT_DAYS_PCT_AAVG', 'LDEAL_ACT_DAYS_PCT_TR', 'LDEAL_ACT_DAYS_PCT_TR4','LDEAL_ACT_DAYS_PCT_CURR'], axis=1, inplace=True)
df.drop(['APP_REGISTR_RGN_CODE'], axis=1, inplace=True)
df['APP_KIND_OF_PROP_HABITATION'].fillna('OTHER', inplace=True)
df['APP_POSITION_TYPE'].fillna('OTHER', inplace=True)
df['CLNT_JOB_POSITION_TYPE'].fillna('OTHER', inplace=True)
df['APP_COMP_TYPE'].fillna('OTHER', inplace=True)
df['APP_EMP_TYPE'].fillna('OTHER', inplace=True)
df.loc[df['APP_MARITAL_STATUS'] == 'm', 'APP_MARITAL_STATUS'] = 'M'
df.loc[df['APP_MARITAL_STATUS'] == 'v', 'APP_MARITAL_STATUS'] = 'V'
df.loc[df['APP_MARITAL_STATUS'] == 'w', 'APP_MARITAL_STATUS'] = 'W'
df.loc[df['APP_MARITAL_STATUS'] == 'd', 'APP_MARITAL_STATUS'] = 'D'
df.loc[df['APP_MARITAL_STATUS'] == 't', 'APP_MARITAL_STATUS'] = 'T'
df['APP_MARITAL_STATUS'].fillna('OTHER', inplace=True)
df.loc[df['APP_EDUCATION'] == 'a', 'APP_EDUCATION'] = 'A'
df.loc[df['APP_EDUCATION'] == 'e', 'APP_EDUCATION'] = 'E'
df.loc[df['APP_EDUCATION'] == 'i', 'APP_EDUCATION'] = 'I'
df.loc[df['APP_EDUCATION'] == 's', 'APP_EDUCATION'] = 'S'
df.loc[df['APP_EDUCATION'] == 'h', 'APP_EDUCATION'] = 'H'
df['APP_EDUCATION'].fillna('OTHER', inplace=True)
df.loc[df['CLNT_TRUST_RELATION'] == 'Друг', 'CLNT_TRUST_RELATION'] = 'FRIEND'
df.loc[df['CLNT_TRUST_RELATION'] == 'Мать', 'CLNT_TRUST_RELATION'] = 'MOTHER'
df.loc[df['CLNT_TRUST_RELATION'] == 'Близкий ро', 'CLNT_TRUST_RELATION'] = 'RELATIVE'
df.loc[df['CLNT_TRUST_RELATION'] == 'Брат', 'CLNT_TRUST_RELATION'] = 'BROTHER'
df.loc[df['CLNT_TRUST_RELATION'] == 'Отец', 'CLNT_TRUST_RELATION'] = 'FATHER'
df.loc[df['CLNT_TRUST_RELATION'] == 'Сестра', 'CLNT_TRUST_RELATION'] = 'SISTER'
df.loc[df['CLNT_TRUST_RELATION'] == 'Дочь', 'CLNT_TRUST_RELATION'] = 'DAUGHTER'
df.loc[df['CLNT_TRUST_RELATION'] == 'Сын', 'CLNT_TRUST_RELATION'] = 'SON'
df.loc[df['CLNT_TRUST_RELATION'] == 'Дальний ро', 'CLNT_TRUST_RELATION'] = 'RELATIVE'
df.loc[df['CLNT_TRUST_RELATION'] == 'Муж', 'CLNT_TRUST_RELATION'] = 'HUSBAND'
df.loc[df['CLNT_TRUST_RELATION'] == 'мать', 'CLNT_TRUST_RELATION'] = 'MOTHER'
df.loc[df['CLNT_TRUST_RELATION'] == 'Жена', 'CLNT_TRUST_RELATION'] = 'WIFE'
df['CLNT_TRUST_RELATION'].fillna('OTHER', inplace=True)
df['AMOUNT_RUB_CLO_PRC'].fillna(0.0, inplace=True)
df['CLNT_SALARY_VALUE'].fillna(df['CLNT_SALARY_VALUE'].mean(), inplace=True)
df['DEAL_YWZ_IR_MAX'].fillna(df['DEAL_YWZ_IR_MAX'].mean(), inplace=True)
df['APP_MARITAL_STATUS'].fillna('OTHER', inplace=True)
df['LDEAL_TENOR_MAX'].value_counts(dropna=False)

NaN      347189
24.0       1316
36.0       1164
12.0       1066
6.0         642
          ...  
75.0          1
123.0         1
81.0          1
69.0          1
158.0         1
Name: LDEAL_TENOR_MAX, Length: 80, dtype: int64

In [14]:
for column in df.columns:
    if (df[column].dtype == np.float64):
        continue
    df[column] = LabelEncoder().fit_transform(df[column])

In [15]:
y = df['TARGET']
X = df.drop(['TARGET'], axis=1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

In [20]:
model = Sequential()
model.add(Dense(128, input_dim=93, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(rate=0.2))
model.add(Dense(64, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(rate=0.2))
model.add(Dense(64, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(rate=0.2))
model.add(Dense(1, activation='sigmoid'))

In [21]:
model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics=['accuracy'])

In [22]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1000)

Epoch 1/10
285/285 [==============================] - 2s 6ms/step - loss: 439.2450 - accuracy: 0.8513 - val_loss: 3.4047 - val_accuracy: 0.9187
Epoch 2/10
285/285 [==============================] - 2s 5ms/step - loss: 9.6639 - accuracy: 0.9060 - val_loss: 0.7540 - val_accuracy: 0.9190
Epoch 3/10
285/285 [==============================] - 2s 7ms/step - loss: 2.1020 - accuracy: 0.9161 - val_loss: 0.3239 - val_accuracy: 0.9190
Epoch 4/10
285/285 [==============================] - 2s 7ms/step - loss: 0.9704 - accuracy: 0.9177 - val_loss: 0.2874 - val_accuracy: 0.9190
Epoch 5/10
285/285 [==============================] - 2s 7ms/step - loss: 0.5237 - accuracy: 0.9181 - val_loss: 0.2824 - val_accuracy: 0.9190
Epoch 6/10
285/285 [==============================] - 2s 6ms/step - loss: 0.4619 - accuracy: 0.9182 - val_loss: 0.2813 - val_accuracy: 0.9190
Epoch 7/10
285/285 [==============================] - 2s 6ms/step - loss: 0.4040 - accuracy: 0.9183 - val_loss: 0.2812 - val_accuracy: 0.9190
Epoc